# Sioux Falls example

## File paths

In [1]:
fldr = 'D:/release/Sample models/sioux_falls_2020_02_15'
proj_name = 'SiouxFalls.sqlite'

# remove the comments for the lines below to run the Chicago model example instead
# fldr = 'D:/release/Sample models/Chicago_2020_02_15'
# proj_name = 'chicagomodel.sqlite'

dt_fldr = '0_tntp_data'
prj_fldr = '1_project'
skm_fldr = '2_skim_results'
assg_fldr = '4_assignment_results'
dstr_fldr = '5_distribution_results'
frcst_fldr = '6_forecast'
ftr_fldr = '7_future_year_assignment'

## Some logging

In [2]:
import aequilibrae
from os.path import join
from aequilibrae import Parameters

p = Parameters()
p.parameters['system']['logging_directory'] =  fldr
p.write_back()

In [3]:
# To make sure the logging will go where it should, we reload aequilibrae
from importlib import reload
reload(aequilibrae)

<module 'aequilibrae' from 'D:\\src\\aequilibrae\\aequilibrae\\__init__.py'>

## Opening the project

In [4]:
# Imports
from aequilibrae.project import Project

In [5]:
project = Project()
project.load(join(fldr, prj_fldr, proj_name))

## Path computation

In [6]:
# imports
from aequilibrae.paths import PathResults, path_computation

In [7]:
# we build all graphs
project.network.build_graphs()
# We get warnings that several fields in the project are filled with NaNs.  Which is true, but we won't use those fields

D:\src\aequilibrae\aequilibrae\project\network\network.py:276: UserWarning: Fields were removed form Graph for being non-numeric: modes,link_type
  warn(f'Fields were removed form Graph for being non-numeric: {",".join(removed_fields)}')
D:\src\aequilibrae\aequilibrae\paths\graph.py:351: UserWarning: Field name has at least one NaN value.  Your computation may be compromised
  warn(f'Field {i} has at least one NaN value.  Your computation may be compromised')
D:\src\aequilibrae\aequilibrae\paths\graph.py:351: UserWarning: Field lanes has at least one NaN value.  Your computation may be compromised
  warn(f'Field {i} has at least one NaN value.  Your computation may be compromised')


In [8]:
# we grab the graph for cars
graph = project.network.graphs['c']

# let's say we want to minimize distance
graph.set_graph('distance')

# And will skim time and distance while we are at it
graph.set_skimming(['free_flow_time', 'distance'])

# And we will allow paths to be compute going through other centroids/centroid connectors
# required for the Sioux Falls network, as all nodes are centroids
graph.set_blocked_centroid_flows(False)

# instantiate a path results object and prepare it to work with the graph
res = PathResults()
res.prepare(graph)

# compute a path from node 2 to 13
path_computation(2, 13, graph, res)

In [9]:
# We can get the sequence of nodes we traverse
res.path_nodes

array([ 2,  1,  3, 12, 13], dtype=int64)

In [10]:
# We can get the link sequence we traverse
res.path

array([ 3,  2,  7, 37], dtype=int64)

In [11]:
# We can get the mileposts for our sequence of nodes
res.milepost

array([ 0.,  6., 10., 14., 17.])

In [12]:
# And We can the skims for our tree
res.skims

array([[ 6.,  6.,  0.],
       [ 0.,  0.,  0.],
       [10., 10.,  0.],
       [11., 11.,  0.],
       [ 9.,  9.,  0.],
       [ 5.,  5.,  0.],
       [10., 10.,  0.],
       [ 7.,  7.,  0.],
       [14., 14.,  0.],
       [16., 16.,  0.],
       [17., 17.,  0.],
       [14., 14.,  0.],
       [17., 17.,  0.],
       [21., 21.,  0.],
       [19., 19.,  0.],
       [12., 12.,  0.],
       [14., 14.,  0.],
       [12., 12.,  0.],
       [16., 16.,  0.],
       [16., 16.,  0.],
       [22., 22.,  0.],
       [21., 21.,  0.],
       [23., 23.,  0.],
       [21., 21.,  0.],
       [ 0.,  0.,  0.]])

In [13]:
# If we want to compute the path for a different destination and same origin, we can just do this
# It is way faster when you have large networks
res.update_trace(4)

In [14]:
res.path_nodes

array([2, 6, 5, 4], dtype=int64)

## Skimming

In [15]:
from aequilibrae.matrix import AequilibraeData, AequilibraeMatrix
from aequilibrae.paths import NetworkSkimming, SkimResults

In [16]:
# from before
project = Project()
project.load(join(fldr, prj_fldr, proj_name))
project.network.build_graphs()

graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_blocked_centroid_flows(False)

In [17]:
# setup the object result
res = SkimResults()
res.prepare(graph)

In [18]:
# And run the skimming
skm = NetworkSkimming(graph, res)
skm.execute()

In [19]:
# The result is an AequilibraEMatrix object
skims = res.skims

# We can export to AEM and OMX
skims.export(join(fldr, skm_fldr, 'skimming_on_time.aem'))
skims.export(join(fldr, skm_fldr, 'skimming_on_time.omx'))

# Traffic assignment with skimming

In [20]:
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae import logger
import logging

In [21]:
# from before
project = Project()
project.load(join(fldr, prj_fldr, proj_name))
project.network.build_graphs()

graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph.set_blocked_centroid_flows(False)

In [22]:
import sys
# Because assignment takes a long time, we want the log to be shown here
stdout_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s;%(name)s;%(levelname)s ; %(message)s")
stdout_handler.setFormatter(formatter)
logger.addHandler(stdout_handler)

In [23]:
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))
demand.computational_view(['matrix']) # We will only assign one user class stored as 'matrix' inside the OMX file

assig = TrafficAssignment()

# Creates the assignment class
assigclass = TrafficClass(graph, demand)


# The first thing to do is to add at list of traffic classes to be assigned
assig.set_classes([assigclass])

assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"}) # And its parameters

assig.set_capacity_field("capacity") # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("free_flow_time")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 1000
assig.rgap_target = 0.00001

assig.execute() # we then execute the assignment

2020-04-06 19:17:14,857;aequilibrae;INFO ; bfw Assignment STATS
2020-04-06 19:17:14,858;aequilibrae;INFO ; Iteration, RelativeGap, stepsize
2020-04-06 19:17:14,966;aequilibrae;INFO ; 1,inf,1.0
2020-04-06 19:17:15,075;aequilibrae;INFO ; 2,0.8485503509703024,0.36497345609427145
2020-04-06 19:17:15,198;aequilibrae;INFO ; 3,0.3813926225800203,0.2298356924660528
2020-04-06 19:17:15,311;aequilibrae;INFO ; 4,0.19621277462606984,0.18591312145268074
2020-04-06 19:17:15,420;aequilibrae;INFO ; 5,0.09069073200924213,0.7090816523174254
2020-04-06 19:17:15,540;aequilibrae;INFO ; 6,0.20600048221061426,0.1229016022154401
2020-04-06 19:17:15,686;aequilibrae;INFO ; 7,0.06710568925282254,0.38638656717489844
2020-04-06 19:17:15,798;aequilibrae;INFO ; 8,0.10307514154369488,0.1093055036410267
2020-04-06 19:17:15,940;aequilibrae;INFO ; 9,0.04222147191362779,0.2487805192125393
2020-04-06 19:17:16,097;aequilibrae;INFO ; 10,0.05926435464772421,0.15904810628271004
2020-04-06 19:17:16,206;aequilibrae;INFO ; 11,0.

### Save outputs


In [24]:
# Convergence report is easy to see
import pandas as pd
convergence_report = pd.DataFrame(assig.assignment.convergence_report)
convergence_report.head()

,iteration,rgap,alpha,warnings,beta0,beta1,beta2
0,1,inf,1.000000,,1.000000,0.000000,0.000000
1,2,0.848550,0.364973,,1.000000,0.000000,0.000000
2,3,0.381393,0.229836,,1.000000,0.000000,0.000000
3,4,0.196213,0.185913,,0.959771,0.040229,0.000000
4,5,0.090691,0.709082,,0.687640,0.286705,0.025654


In [25]:
# The link flows are easy to export.
# we do so for csv and AequilibraEData
assigclass.results.save_to_disk(join(fldr, assg_fldr, 'link_flows_c.csv'), output="loads")
assigclass.results.save_to_disk(join(fldr, assg_fldr, 'link_flows_c.aed'), output="loads")

In [26]:
# the skims are easy to get.

# The blended one are here
avg_skims = assigclass.results.skims

# The ones for the last iteration are here
last_skims = assigclass._aon_results.skims

# Assembling a single final skim file can be done like this
# We will want only the time for the last iteration and the distance averaged out for all iterations
kwargs = {'file_name': join(fldr,assg_fldr, 'skims.aem'),
          'zones': graph.num_zones,
          'matrix_names': ['time_final', 'distance_blended']}

# Create the matrix file
out_skims = AequilibraeMatrix()
out_skims.create_empty(**kwargs)
out_skims.index[:] = avg_skims.index[:]

# Transfer the data
 # The names of the skims are the name of the fields
out_skims.matrix['time_final'][:,:] = last_skims.matrix['free_flow_time'][:,:]
# It is CRITICAL to assign the matrix values using the [:,:]
out_skims.matrix['distance_blended'][:,:] = avg_skims.matrix['distance'][:,:]

out_skims.matrices.flush() # Make sure that all data went to the disk

# Export to OMX as well
out_skims.export(join(fldr,assg_fldr, 'skims.omx'))

    

# Trip distribution

### Calibration

We will calibrate synthetic gravity models using the skims for TIME that we just generated

In [27]:
import numpy as np
from aequilibrae.distribution import GravityCalibration
from aequilibrae.matrix import AequilibraeMatrix

In [28]:
# We need the demand
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))

# And the skims
imped = AequilibraeMatrix()
imped.load(join(fldr,assg_fldr, 'skims.aem'))

In [29]:
# But before using the data, let's get some impedance for the intrazonals
# Let's assume it is 75% of the closest zone

# If we run the code below more than once, we will be overwriting the diagonal values with non-sensical data
# so let's zero it first
np.fill_diagonal(imped.matrix['time_final'], 0)

# We compute it with a little bit of NumPy magic
intrazonals = np.amin(imped.matrix['time_final'], where=imped.matrix['time_final']>0, initial=imped.matrix['time_final'].max(), axis=1)
intrazonals *= 0.75

# Then we fill in the impedance matrix
np.fill_diagonal(imped.matrix['time_final'], intrazonals)


In [30]:
# We set the matrices forbeing used in computation
imped.computational_view(['time_final'])
demand.computational_view(['matrix'])

In [31]:
from math import log10, floor
def plot_tlfd(demand, skim, name):
    import matplotlib.pyplot as plt
    b = floor(log10(skim.shape[0]) * 10)
    n, bins, patches = plt.hist(np.nan_to_num(skim.flatten(),0), bins = b, weights=np.nan_to_num(demand.flatten()), density=False, facecolor='g', alpha=0.75)

    plt.xlabel('Trip length')
    plt.ylabel('Probability')
    plt.title('Trip-length frequency distribution')
    plt.savefig(name, format="png")
    plt.clf()

In [32]:
for function in ['power', 'expo']:
    model = GravityCalibration(matrix=demand, impedance=imped, function=function, nan_as_zero=True)
    model.calibrate()
    
    # we save the model
    model.model.save(join(fldr, dstr_fldr, f'{function}_model.mod'))
    
    # We save a trip length frequency distribution image
    plot_tlfd(model.result_matrix.matrix_view, imped.matrix_view,join(fldr, dstr_fldr, f'{function}_tfld.png') )
    
    # We can save the result of applying the model as well
    # we can also save the calibration report
    with open(join(fldr, dstr_fldr, f'{function}_convergence.log'), 'w') as otp:
        for r in  model.report:
            otp.write(r+'\n')

<Figure size 432x288 with 0 Axes>

In [33]:
# We save a trip length frequency distribution image
plot_tlfd(demand.matrix_view, imped.matrix_view,join(fldr, dstr_fldr, 'demand_tfld.png') )

<Figure size 432x288 with 0 Axes>

# Forecast

* We create a set of *'future'* vectors using some random growth factors
* We apply the model for inverse power, as the TFLD seems to be a better fit for the actual one

In [34]:
from aequilibrae.distribution import Ipf, GravityApplication, SyntheticGravityModel, Ipf
from aequilibrae.matrix import AequilibraeData, AequilibraeMatrix
import numpy as np

In [35]:
# We compute the vectors from our matrix
mat = AequilibraeMatrix()

mat.load(join(fldr, dt_fldr, 'demand.omx'))
mat.computational_view()
origins = np.sum(mat.matrix_view, axis=1)
destinations = np.sum(mat.matrix_view, axis=0)

args = {'file_path':join(fldr,  frcst_fldr, 'synthetic_future_vector.aed'),
        "entries": mat.zones, 
        "field_names": ["origins", "destinations"],
    "data_types": [np.float64, np.float64], 
        "memory_mode": False}

vectors = AequilibraeData()
vectors.create_empty(**args)

vectors.index[:] =mat.index[:]

# Then grow them with some random growth between 0 and 10% - Plus balance them
vectors.origins[:] = origins * (1+ np.random.rand(vectors.entries)/10)
vectors.destinations[:] = destinations * (1+ np.random.rand(vectors.entries)/10)
vectors.destinations *= vectors.origins.sum()/vectors.destinations.sum()

In [36]:
# Impedance 
imped = AequilibraeMatrix()
imped.load(join(fldr,assg_fldr, 'skims.aem'))
imped.computational_view(['time_final'])

# We want the main diagonal to be zero
np.fill_diagonal(imped.matrix_view, np.nan)

In [37]:
for function in ['power', 'expo']:
    model = SyntheticGravityModel()
    model.load(join(fldr, dstr_fldr, f'{function}_model.mod'))

    outmatrix = join(fldr,frcst_fldr, f'demand_{function}_model.aem') 
    apply = GravityApplication()
    args = {"impedance": imped,
            "rows": vectors,
            "row_field": "origins",
            "model": model,
            "columns": vectors,
            "column_field": "destinations",
            "output": outmatrix,
            "nan_as_zero":True
            }

    gravity = GravityApplication(**args)
    gravity.apply()

    #We get the output matrix and save it to OMX too
    resm = AequilibraeMatrix()
    resm.load(outmatrix)
    resm.export(join(fldr,frcst_fldr, f'demand_{function}_model.omx'))

### We now run IPF for the future vectors

In [38]:
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))
demand.computational_view()

args = {'matrix': demand,
        'rows': vectors,
        'columns': vectors,
        'column_field': "destinations",
        'row_field': "origins",
        'nan_as_zero': True}

ipf = Ipf(**args)
ipf.fit()

output = AequilibraeMatrix()
output.load(ipf.output.file_path)

output.export(join(fldr,frcst_fldr, 'demand_ipf.aem'))
output.export(join(fldr,frcst_fldr, 'demand_ipf.omx'))


# Future traffic assignment

In [39]:
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae import logger
import logging

In [40]:
logger.info('\n\n\n TRAFFIC ASSIGNMENT FOR FUTURE YEAR')

2020-04-06 19:17:51,944;aequilibrae;INFO ; 


 TRAFFIC ASSIGNMENT FOR FUTURE YEAR


In [41]:
# from before
project = Project()
project.load(join(fldr, prj_fldr, proj_name))
project.network.build_graphs()

graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph.set_blocked_centroid_flows(False)

D:\src\aequilibrae\aequilibrae\project\network\network.py:276: UserWarning: Fields were removed form Graph for being non-numeric: modes,link_type
  warn(f'Fields were removed form Graph for being non-numeric: {",".join(removed_fields)}')
D:\src\aequilibrae\aequilibrae\paths\graph.py:351: UserWarning: Field name has at least one NaN value.  Your computation may be compromised
  warn(f'Field {i} has at least one NaN value.  Your computation may be compromised')
D:\src\aequilibrae\aequilibrae\paths\graph.py:351: UserWarning: Field lanes has at least one NaN value.  Your computation may be compromised
  warn(f'Field {i} has at least one NaN value.  Your computation may be compromised')


In [42]:
# Let's use the IPF matrix
demand = AequilibraeMatrix()
demand.load(join(fldr, frcst_fldr, 'demand_ipf.omx'))
demand.computational_view() # There is only one matrix there, so don;t even worry about its core name

assig = TrafficAssignment()

# Creates the assignment class
assigclass = TrafficClass(graph, demand)

# The first thing to do is to add at list of traffic classes to be assigned
assig.set_classes([assigclass])

assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"}) # And its parameters

assig.set_capacity_field("capacity") # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("free_flow_time")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 1000
assig.rgap_target = 0.00001

assig.execute() # we then execute the assignment

2020-04-06 19:17:52,305;aequilibrae;INFO ; bfw Assignment STATS
2020-04-06 19:17:52,306;aequilibrae;INFO ; Iteration, RelativeGap, stepsize
2020-04-06 19:17:52,412;aequilibrae;INFO ; 1,inf,1.0
2020-04-06 19:17:52,523;aequilibrae;INFO ; 2,0.8621636013061809,0.35637756328056774
2020-04-06 19:17:52,634;aequilibrae;INFO ; 3,0.4178112735175096,0.1905118959633645
2020-04-06 19:17:52,745;aequilibrae;INFO ; 4,0.2207501654669293,0.2517357451991883
2020-04-06 19:17:52,856;aequilibrae;INFO ; 5,0.11092078855747255,0.7928883486959472
2020-04-06 19:17:52,969;aequilibrae;INFO ; 6,0.20789256138537235,0.1250370481076385
2020-04-06 19:17:53,081;aequilibrae;INFO ; 7,0.07198179367868165,0.2565052923285979
2020-04-06 19:17:53,192;aequilibrae;INFO ; 8,0.07585645172316004,0.8046536807023875
2020-04-06 19:17:53,305;aequilibrae;INFO ; 9,0.09157482037405977,0.09627048336694917
2020-04-06 19:17:53,416;aequilibrae;INFO ; 10,0.04910608978442712,0.17207964718492155
2020-04-06 19:17:53,528;aequilibrae;INFO ; 11,0.03

In [43]:

# The link flows are easy to export.
# we do so for csv and AequilibraEData
assigclass.results.save_to_disk(join(fldr, ftr_fldr, 'future_link_flows_c.csv'), output="loads")
assigclass.results.save_to_disk(join(fldr, ftr_fldr, 'future_link_flows_c.aed'), output="loads")

# the skims are easy to get.

# The blended one are here
avg_skims = assigclass.results.skims

# The ones for the last iteration are here
last_skims = assigclass._aon_results.skims

# Assembling a single final skim file can be done like this
# We will want only the time for the last iteration and the distance averaged out for all iterations
kwargs = {'file_name': join(fldr,ftr_fldr, 'future_skims.aem'),
          'zones': graph.num_zones,
          'matrix_names': ['time_final', 'distance_blended']}

# Create the matrix file
out_skims = AequilibraeMatrix()
out_skims.create_empty(**kwargs)
out_skims.index[:] = avg_skims.index[:]

# Transfer the data
 # The names of the skims are the name of the fields
out_skims.matrix['time_final'][:,:] = last_skims.matrix['free_flow_time'][:,:]
# It is CRITICAL to assign the matrix values using the [:,:]
out_skims.matrix['distance_blended'][:,:] = avg_skims.matrix['distance'][:,:]

out_skims.matrices.flush() # Make sure that all data went to the disk

# Export to OMX as well
out_skims.export(join(fldr,ftr_fldr, 'future_skims.omx'))
